In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib as mpl

# style your matplotlib
mpl.style.use("seaborn-darkgrid")
# run this block

In [2]:
from tqdm import tqdm

In [3]:
files=os.listdir("../input/face-recognition-30/dataset/")
files

['Courteney_Cox',
 'arnold_schwarzenegger',
 'bhuvan_bam',
 'hardik_pandya',
 'David_Schwimmer',
 'Matt_LeBlanc',
 'Simon_Helberg',
 'scarlett_johansson',
 'Pankaj_Tripathi',
 'Matthew_Perry',
 'sylvester_stallone',
 'messi',
 'Jim_Parsons',
 'random_person',
 'Lisa_Kudrow',
 'mohamed_ali',
 'brad_pitt',
 'ronaldo',
 'virat_kohli',
 'angelina_jolie',
 'Kunal_Nayya',
 'manoj_bajpayee',
 'Sachin_Tendulka',
 'Jennifer_Aniston',
 'dhoni',
 'pewdiepie',
 'aishwarya_rai',
 'Johnny_Galeck',
 'ROHIT_SHARMA',
 'suresh_raina']

In [4]:
image_array=[]  # it's a list later i will convert it to array
label_array=[]
path="../input/face-recognition-30/dataset/"
# loop through each sub-folder in train
for i in range(len(files)):
    # files in sub-folder
    file_sub=os.listdir(path+files[i])

    for k in tqdm(range(len(file_sub))):
        try:
            img=cv2.imread(path+files[i]+"/"+file_sub[k])
            img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            img=cv2.resize(img,(96,96))
            image_array.append(img)
            label_array.append(i)
        except:
            pass

100%|██████████| 319/319 [00:01<00:00, 282.58it/s]


In [5]:
import gc
gc.collect()

666

In [6]:

image_array=np.array(image_array)/255.0
label_array=np.array(label_array)

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(image_array,label_array,test_size=0.15)

In [8]:
from keras import layers,callbacks,utils,applications,optimizers
from keras.models import Sequential,Model,load_model

In [9]:
len(files)

30

In [10]:
model=Sequential()
# I will use MobileNetV2 as an pretrained model 
pretrained_model=applications.MobileNetV2(input_shape=(96,96,3),include_top=False,
                                         weights="imagenet")
model.add(pretrained_model)
model.add(layers.GlobalAveragePooling2D())
# add dropout to increase accuracy by not overfitting
model.add(layers.Dropout(0.3))
# add dense layer as final output
model.add(layers.Dense(1))
model.summary()

9412608/9406464 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_96 (Functio (None, 3, 3, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 2,225,153
Non-trainable params: 34,112
_________________________________________________________________


In [11]:
model.compile(optimizer="adam",loss="mean_squared_error",metrics=["mae"])

In [12]:
# creating a chechpoint to save model at best accuarcy

ckp_path="trained_model/model"
model_checkpoint=tf.keras.callbacks.ModelCheckpoint(filepath=ckp_path,
                                                   monitor="val_mae",
                                                   mode="auto",
                                                   save_best_only=True,
                                                   save_weights_only=True)

In [13]:
# create a lr reducer which decrease learning rate when accuarcy does not increase
reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(factor=0.9,monitor="val_mae",
                                             mode="auto",cooldown=0,
                                             patience=5,verbose=1,min_lr=1e-6)
# patience : wait till 5 epoch
# verbose : show accuracy every 1 epoch
# min_lr=minimum learning rate
#

In [14]:
EPOCHS=300
BATCH_SIZE=64

history=model.fit(X_train,
                 Y_train,
                 validation_data=(X_test,Y_test),
                 batch_size=BATCH_SIZE,
                 epochs=EPOCHS,
                 callbacks=[model_checkpoint,reduce_lr]
                 )

Epoch 1/300
212/212 [==============================] - 23s 74ms/step - loss: 68.4382 - mae: 6.1899 - val_loss: 436.4095 - val_mae: 19.4031
Epoch 2/300
212/212 [==============================] - 14s 66ms/step - loss: 18.7415 - mae: 3.1299 - val_loss: 224.2123 - val_mae: 13.1698
Epoch 3/300
212/212 [==============================] - 14s 66ms/step - loss: 10.6919 - mae: 2.2561 - val_loss: 190.3877 - val_mae: 11.8455
Epoch 4/300
212/212 [==============================] - 14s 66ms/step - loss: 8.7893 - mae: 1.9682 - val_loss: 203.1610 - val_mae: 12.4595
Epoch 5/300
212/212 [==============================] - 14s 66ms/step - loss: 8.0304 - mae: 1.9031 - val_loss: 140.1621 - val_mae: 10.0426
Epoch 6/300
212/212 [==============================] - 14s 66ms/step - loss: 5.8401 - mae: 1.5403 - val_loss: 169.7178 - val_mae: 11.4854
Epoch 7/300
212/212 [==============================] - 14s 66ms/step - loss: 5.4382 - mae: 1.4565 - val_loss: 57.0441 - val_mae: 5.8880
Epoch 8/300
212/212 [============

In [15]:
model.load_weights(ckp_path)

In [16]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [17]:
prediction_val=model.predict(X_test,batch_size=BATCH_SIZE)

In [18]:
prediction_val[:20]

array([[12.824087 ],
       [13.945617 ],
       [10.00906  ],
       [15.964796 ],
       [ 6.268346 ],
       [26.009563 ],
       [ 6.1312485],
       [ 5.951178 ],
       [18.081257 ],
       [ 1.9568746],
       [ 3.9703596],
       [15.119955 ],
       [ 2.3849247],
       [ 7.2983627],
       [15.854818 ],
       [13.040954 ],
       [ 8.112918 ],
       [22.709955 ],
       [13.099166 ],
       [ 1.1193055]], dtype=float32)

In [19]:
Y_test[:20]

array([13, 14, 10, 16,  6, 26,  6,  6, 18,  2,  4, 15,  2,  7, 17,  2,  8,
       23, 26,  1])